##### installs & imports

In [40]:
!pip install datasets

In [41]:
!pip install PyPDF2

In [ ]:
import pandas as pd
import re
import json

##### dataset 1 : huggingface

In [43]:
from datasets import load_dataset

# 'aueb-nlp/ecthr_cases' 데이터셋 로드
dataset = load_dataset("aueb-nlp/ecthr_cases")

# 데이터셋 확인 (train/validation/test)
print(dataset)

# 첫 번째 예시 출력
print(dataset['train'][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

ecthr_cases.py:   0%|          | 0.00/8.61k [00:00<?, ?B/s]

The repository for aueb-nlp/ecthr_cases contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/aueb-nlp/ecthr_cases.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['facts', 'labels', 'silver_rationales', 'gold_rationales'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['facts', 'labels', 'silver_rationales', 'gold_rationales'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['facts', 'labels', 'silver_rationales', 'gold_rationales'],
        num_rows: 1000
    })
})
{'facts': ['11.  At the beginning of the events relevant to the application, K. had a daughter, P., and a son, M., born in 1986 and 1988 respectively. P.’s father is X and M.’s father is V. From March to May 1989 K. was voluntarily hospitalised for about three months, having been diagnosed as suffering from schizophrenia. From August to November 1989 and from December 1989 to March 1990, she was again hospitalised for periods of about three months on account of this illness. In 1991 she was hospitalised for less than a week, diagnosed as suffering from an atypical and undefinable ps

In [44]:
df_train = pd.DataFrame(dataset['train'])
df_train.head()

,facts,labels,silver_rationales,gold_rationales
0,[11. At the beginning of the events relevant ...,"[13, 8]","[1, 13, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30...",[]
1,[9. The applicant is the monarch of Liechtens...,"[14, P1-1, 6]","[3, 6]",[]
2,[9. In June 1949 plots of agricultural land o...,[6],"[4, 5]",[]
3,"[8. In 1991 Mr Dušan Slobodník, a research wo...","[14, 10, 9]",[27],[]
4,"[9. The applicant is an Italian citizen, born...","[14, 6]",[4],[]


In [45]:
len(dataset['train']), len(dataset['validation']), len(dataset['test'])

(9000, 1000, 1000)

##### dataset2 : kaggle echr, ecthr

In [ ]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)


In [10]:
train_data = load_jsonl('../echr/train.jsonl')
dev_data = load_jsonl('../echr/dev.jsonl')
test_data = load_jsonl('../echr/test.jsonl')

In [ ]:
# 현재는 두 파일이 동일해서 제거한 상태
# Load the datasets
ecthr_train_data = load_jsonl('ecthr/train.jsonl')
ecthr_dev_data = load_jsonl('ecthr/dev.jsonl')
ecthr_test_data = load_jsonl('ecthr/test.jsonl')

(dev_data != ecthr_dev_data).sum().sum(), (train_data != ecthr_train_data).sum().sum(), (test_data != ecthr_test_data).sum().sum()

##### EDA

In [54]:
len(train_data), len(test_data), len(dev_data)

(9000, 1000, 1000)

In [55]:
train_data.head()

,case_id,case_no,title,judgment_date,facts,applicants,defendants,allegedly_violated_articles,violated_articles,court_assessment_references,silver_rationales,gold_rationales
0,001-59587,25702/94,CASE OF K. AND T. v. FINLAND,2001-07-12,[11. At the beginning of the events relevant ...,"[K., T.]",[FINLAND],"[13, 8]",[8],"{'8': ['12', '140', '155', '156', '157', '158'...","[1, 13, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30...",[]
1,001-59591,42527/98,CASE OF PRINCE HANS-ADAM II OF LIECHTENSTEIN v...,2001-07-12,[9. The applicant is the monarch of Liechtens...,[PRINCE HANS-ADAM II OF LIECHTENSTEIN],[GERMANY],"[14, P1-1, 6]",[],"{'6': ['12', '15', '24', '25', '26', '27', '28...","[3, 6]",[]
2,001-59590,33071/96,CASE OF MALHOUS v. THE CZECH REPUBLIC,2001-07-12,[9. In June 1949 plots of agricultural land o...,[MALHOUS],[CZECH REPUBLIC],[6],[6],"{'6': ['13', '14', '35', '40', '41', '42', '43...","[4, 5]",[]
3,001-59588,29032/95,CASE OF FELDEK v. SLOVAKIA,2001-07-12,"[8. In 1991 Mr Dušan Slobodník, a research wo...",[FELDEK],[SLOVAKIA],"[14, 10, 9]",[10],{'10': ['35']},[27],[]
4,001-59589,44759/98,CASE OF FERRAZZINI v. ITALY,2001-07-12,"[9. The applicant is an Italian citizen, born...",[FERRAZZINI],[ITALY],"[14, 6]",[],"{'6': ['13', '14', '35', '40', '41', '42', '43...",[4],[]


In [64]:
(train_data[['facts', 'allegedly_violated_articles', 'silver_rationales', 'gold_rationales' ]].rename(columns={'allegedly_violated_articles':'labels'}) != df_train).sum().sum()

np.int64(0)

##### article 정보 추출

In [70]:
unique_viol_articles = train_data['allegedly_violated_articles'].apply(pd.Series).stack().unique()
unique_viol_articles.sort()
unique_viol_articles

array(['10', '11', '12', '13', '14', '15', '17', '18', '2', '3', '34',
       '38', '39', '4', '46', '5', '6', '7', '8', '9', 'P1-1', 'P1-2',
       'P1-3', 'P12-1', 'P3-1', 'P4-2', 'P4-4', 'P6-3', 'P7-1', 'P7-2',
       'P7-3', 'P7-4', 'P7-5'], dtype=object)

In [ ]:
from PyPDF2 import PdfReader

# Load the PDF
pdf_path = "echr_Convention-texts.pdf"
reader = PdfReader(pdf_path)

# Extract text from all pages
full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

# Target articles
target_articles = unique_viol_articles

In [74]:
# Protocol 블록
protocol_blocks = {
    '': r'(?s)(.*?)Protocol\n',  # Convention 본문
    'P1': r'(?s)Protocol\n(.*?)(?=Protocol No\. 4)',
    'P4': r'(?s)Protocol No\. 4(.*?)(?=Protocol No\. 6)',
    'P6': r'(?s)Protocol No\. 6(.*?)(?=Protocol No\. 7)',
    'P7': r'(?s)Protocol No\. 7(.*?)(?=Protocol No\. 12)',
    'P12': r'(?s)Protocol No\. 12(.*?)(?=Protocol No\. 13)',
}


# Article 추출 패턴 생성
def build_article_pattern(article_number):
    return rf'ARTICLE {article_number}\n+(.+?)(?=(ARTICLE|SECTION|PROTOCOL)[\s\d])'


# 결과 저장
extracted_articles = {}

# 블록 단위로 탐색
for prefix, block_pattern in protocol_blocks.items():
    match = re.search(block_pattern, full_text, re.IGNORECASE)
    if not match:
        continue
    block_text = match.group(1)

    for target in [a for a in target_articles if a.startswith(prefix)]:
        if (prefix == "") & (target[0] == "P"):
            continue
        # print(prefix, target)
        article_num = target.split('-')[-1] if '-' in target else target
        pattern = build_article_pattern(article_num)
        match_article = re.search(pattern, block_text, re.DOTALL)
        if match_article:
            extracted_articles[target] = match_article.group(1).strip()


In [75]:
# 첫 \n 제외 특수기호 제거
for key in extracted_articles:
    parts = extracted_articles[key].split('\n', 1)
    if len(parts) == 2:
        first, rest = parts
        rest_cleaned = rest.replace('\n', ' ')
        extracted_articles[key] = first + '\n' + rest_cleaned
    else:
        extracted_articles[key] = extracted_articles[key]

    extracted_articles[key] = extracted_articles[key].replace('\t', ' ')
    extracted_articles[key] = extracted_articles[key].replace('  ', ' ')

# 페이지를 넘어갈 경우, 문장 안에 페이지 번호가 들어간 경우들 찾아서 제거
pattern = re.compile(r'\b\d{1,2}\s+\d{1,2}\b')

for key, text in extracted_articles.items():
    matches = pattern.findall(text)
    if matches:
        print(key, matches)

        # 해당 패턴 제거
        cleaned_text = pattern.sub('', text)

        # 중복 공백 정리 및 양쪽 공백 제거
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        extracted_articles[key] = cleaned_text


extracted_articles

15 ['14 15']
39 ['24 25']
4 ['8 9']
6 ['10 11']
9 ['12 13']
P1-1 ['34 35']
P4-4 ['38 39']
P7-3 ['46 47']


{'10': 'Freedom of expression\n1. Everyone has the right to freedom of expression. This right shall include freedom to hold opinions and to receive and impart information and ideas without interference by public authority and regardless of frontiers. This Article shall not prevent States from requiring the licensing of broadcasting, television or cinema enterprises. 2. The exercise of these freedoms, since it carries with it duties and responsibilities, may be subject to such formalities, conditions, restrictions or penalties as are prescribed by law and are necessary in a democratic society, in the interests of national security, territorial integrity or public safety, for the prevention of disorder or crime, for the protection of health or morals, for the protection of the reputation or rights of others, for preventing the disclosure of information received in confidence, or for maintaining the authority and impartiality of the judiciary.',
 '11': 'Freedom of assembly and association

In [76]:
json.dump(extracted_articles, open('article_text.json', 'w'), indent=4)